In [1]:
import sys
sys.path.append('../..')
import torchvision 
from torchvision import transforms
from torch.utils.data import DataLoader
import torch as ch
import torch.nn as nn
import math
import matplotlib.pyplot as plt
import cox
from cox.utils import Parameters
import cox.store as store
from cox.readers import CollectionReader
import seaborn as sns
import pickle
import os

from delphi import train
from delphi.imagenet_models import vgg11
from delphi.utils import model_utils
from delphi.utils.datasets import ImageNet
import delphi.utils.constants as consts
import delphi.utils.data_augmentation as da
from delphi.utils.helpers import setup_store_with_metadata

# set environment variable so that stores can create output files
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

# Constants

In [2]:
BASE_CLASSIFIER_PATH = '/home/gridsan/stefanou/VGG-11-IMAGENET/'

# Experiment Hyperparameters

In [3]:
args = Parameters({ 
    'epochs': 150,
    'num_workers': 8, 
    'batch_size': 128, 
    'lr': 1e-1, 
    'momentum': .9, 
    'weight_decay': 5e-4, 
    'save_ckpt_iters': 50,
    'should_save_ckpt': True,
    'log_iters': 1,
    'custom_lr_multiplier': consts.COSINE, 
    'validation_split': .8,
    'shuffle': True,
})
if ch.cuda.is_available(): 
    args.__setattr__('device', 'cuda')
args

{
  "epochs": 150,
  "num_workers": 8,
  "batch_size": 128,
  "lr": 0.1,
  "momentum": 0.9,
  "weight_decay": 0.0005,
  "save_ckpt_iters": 50,
  "should_save_ckpt": true,
  "log_iters": 1,
  "custom_lr_multiplier": "cosine",
  "validation_split": 0.8,
  "shuffle": true,
  "device": "cuda"
}

# Setup and Explore Dataset

In [6]:
ds = ImageNet(data_path='/home/gridsan/stefanou/')
model, _ = model_utils.make_and_restore_model(arch='vgg11', dataset=ds)
train_loader, val_loader = ds.make_loaders(batch_size=128, workers=8)

==> Preparing dataset imagenet..


In [5]:
labels = {i for i in range(10)}
for image, target in train_loader: 
    for j in range(len(target)): 
        im, targ = image[j], target[j]
        if targ.item() in labels: 
            print("label: {}".format(da.CIFAR_LABELS[targ.item()]))
            plt.imshow(transforms.ToPILImage(mode="RGB")(im.squeeze(0)))
            plt.show()
            labels.remove(targ.item())
        if len(labels) == 0: 
            break

UnidentifiedImageError: Caught UnidentifiedImageError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/gridsan/stefanou/.local/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/gridsan/stefanou/.local/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/gridsan/stefanou/.local/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "../../delphi/utils/folder.py", line 130, in __getitem__
    sample = self.loader(path)
  File "../../delphi/utils/folder.py", line 174, in default_loader
    return pil_loader(path)
  File "../../delphi/utils/folder.py", line 158, in pil_loader
    img = Image.open(f)
  File "/home/gridsan/stefanou/.local/lib/python3.8/site-pac

# Setup Store and Train Model

In [6]:
# logging store
out_store = store.Store(BASE_CLASSIFIER_PATH)
setup_store_with_metadata(args, out_store)

Logging in: /home/gridsan/stefanou/VGG-11-CIFAR-10/65173d32-f71f-4bc0-ab73-d632ff2ed808


In [ ]:
# train
final_model = train.train_model(args, model, (train_loader, val_loader), store=out_store, device=args.device)

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

...

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
reader = CollectionReader(BASE_CLASSIFIER_PATH)
metadata = reader.df('metadata')
logs = reader.df('logs')
reader.close()
logs.head()

# Accuracy and Loss Curves

In [ ]:
sns.lineplot(data=logs, x='epoch', y='val_prec1', label='val_prec1', color='b')
ax = sns.lineplot(data=logs, x='epoch', y='train_prec1', label='train_prec1', color='r')
ax.set(xlabel='Epoch', ylabel='Accuracy')
plt.show()

sns.lineplot(data=logs, x='epoch', y='val_loss', label='val_loss', color='b')
ax = sns.lineplot(data=logs, x='epoch', y='train_loss', label='train_loss', color='r')
ax.set(xlabel='Epoch', ylabel='Loss')

# Load in Model

In [11]:
model_utils.make_and_restore_model(arch='vgg11', dataset=ds, resume_path=BASE_CLASSIFIER_PATH + '15033fc8-d094-461c-b5eb-7fef776f23ec/checkpoint.pt.best')

=> loading checkpoint '/home/pstefanou/VGG-11-CIFAR-10/15033fc8-d094-461c-b5eb-7fef776f23ec/checkpoint.pt.best'


RuntimeError: Error(s) in loading state_dict for AttackerModel:
	Missing key(s) in state_dict: "normalizer.new_mean", "normalizer.new_std", "model.features.0.weight", "model.features.0.bias", "model.features.1.weight", "model.features.1.bias", "model.features.1.running_mean", "model.features.1.running_var", "model.features.4.weight", "model.features.4.bias", "model.features.5.weight", "model.features.5.bias", "model.features.5.running_mean", "model.features.5.running_var", "model.features.8.weight", "model.features.8.bias", "model.features.9.weight", "model.features.9.bias", "model.features.9.running_mean", "model.features.9.running_var", "model.features.11.weight", "model.features.11.bias", "model.features.12.weight", "model.features.12.bias", "model.features.12.running_mean", "model.features.12.running_var", "model.features.15.weight", "model.features.15.bias", "model.features.16.weight", "model.features.16.bias", "model.features.16.running_mean", "model.features.16.running_var", "mo

# Test Accuracy

In [24]:
test_store = store.Store(args.out_dir)
results = train.eval_model(args, base_model, test_loader, test_store)
results

{'test_prec1': tensor(91.1700, device='cuda:0'),
 'test_loss': 0.36373048400878905,
 'time': 1.1794376373291016}